<a href="https://colab.research.google.com/github/Arijit02/Sphinx/blob/master/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
import tensorflow as tf
from keras.preprocessing import sequence
from keras.datasets import imdb
import os
import numpy as np

Using TensorFlow backend.


In [0]:
VOCAB_SIZE = 88584
MAX_LEN = 250
BATCH_SIZE = 64

In [4]:
(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

17465344/17464789 [==============================] - 1s 0us/step


In [0]:
train_data = sequence.pad_sequences(train_data,MAX_LEN)         #Making all words in training set equal to 250
test_data = sequence.pad_sequences(test_data,MAX_LEN)           #Making all words in testing set equal to 250

In [0]:
model = tf.keras.Sequential([
      tf.keras.layers.Embedding(VOCAB_SIZE,32),
      tf.keras.layers.LSTM(32),
      tf.keras.layers.Dense(1,'sigmoid')
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

history = model.fit(train_data,train_labels,epochs=10,validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 23s 1ms/sample - loss: 0.4160 - acc: 0.8102 - val_loss: 0.3404 - val_acc: 0.8622
Epoch 2/10
20000/20000 [==============================] - 16s 778us/sample - loss: 0.2398 - acc: 0.9093 - val_loss: 0.2745 - val_acc: 0.8892
Epoch 3/10
20000/20000 [==============================] - 14s 723us/sample - loss: 0.1854 - acc: 0.9324 - val_loss: 0.2743 - val_acc: 0.8900
Epoch 4/10
20000/20000 [==============================] - 14s 718us/sample - loss: 0.1504 - acc: 0.9478 - val_loss: 0.2784 - val_acc: 0.8926
Epoch 5/10
20000/20000 [==============================] - 15s 727us/sample - loss: 0.1300 - acc: 0.9557 - val_loss: 0.2949 - val_acc: 0.8930
Epoch 6/10
20000/20000 [==============================] - 14s 725us/sample - loss: 0.1112 - acc: 0.9617 - val_loss: 0.3141 - val_acc: 0.8934
Epoch 7/10
20000/20000 [==============================] - 14s 718us/sample - loss: 0.0979 - acc: 0.9672 - v

In [9]:
results = model.evaluate(test_data,test_labels)
print(results)

25000/25000 [==============================] - 4s 168us/sample - loss: 0.4709 - acc: 0.8540
[0.4708574286878109, 0.85404]


In [15]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens],MAX_LEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)



[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [13]:
reverse_word_index = {value:key for (key,value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] + " "
  return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [0]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  if result[0]>0.5:
    print("Positive")
  else:
    print("Negative") 

In [20]:
positive_review = "That movie was so awesome! I really loved it and would watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

Positive
Negative
